## Introduction

We first connect our Jupyter Notebook to the database file.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Overview of the Data

In [2]:
%%sql
SELECT * 
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float, is_a_country BOOL DEFAULT 1)"


To get an understanding of what the data and columns look like, we print the first five lines of the database.

In [3]:
%%sql
ALTER TABLE facts
DROP is_a_country

 * sqlite:///factbook.db
Done.


[]

In [4]:
%%sql
SELECT COUNT(*)
  FROM facts

 * sqlite:///factbook.db
Done.


COUNT(*)
261


There are 261 rows in this database, a lot more than the number of official countries.

In [5]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Summary Statistics

Here are the descriptions for some of the columns:
- `name` - the name of the country
- `area` - the country's total area (both land and water)
- `area_land` - the country's land area in square kilometers
- `area_water` - the country's water area in square kilometers
- `population` - the country's population
- `population_growth` - the country's population growth as a percentage
- `birth_rate` - the country's birth rate, or the number of births per year per 1,000 people
- `death_rate` - the country's death rate, or the number of deaths per year per 1,000 people

We start by finding the following:
- minimum population
- maximum population
- minimum population growth
- maximum population growth

In [6]:
%%sql
SELECT min(population) AS min_population,
       max(population) AS max_population,
       min(population_growth) AS min_population_growth,
       max(population_growth) AS max_population_growth
  FROM facts

 * sqlite:///factbook.db
Done.


min_population,max_population,min_population_growth,max_population_growth
0,7256490011,0.0,4.02


The maximum population of 7,256,490,011 people is not currently feasible for one country. We also want to know what country or countries are showing zero population count in our data.

## Cleaning the Data

From later stages, a few observations were made. There are rows for the major oceans, the world in total, and various inhabitable islands with no provided population count. We want to remove these from our analysis.

### World

In [7]:
%%sql
SELECT name
  FROM facts
 WHERE population = (SELECT max(population) 
                       FROM facts)

 * sqlite:///factbook.db
Done.


name
World


The dataset has a row specific to the entire 'World' which explains the 7.26 billion people as its population. We'll want to remove the `World` line to look at the data by country.

### Oceans and Small Inhabitable Islands

In [8]:
%%sql
SELECT *
  FROM facts
 ORDER BY population 
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
198,at,Ashmore and Cartier Islands,5,5,0,None,None,None,None,None
201,cr,Coral Sea Islands,3,3,0,None,None,None,None,None
202,hm,Heard Island and McDonald Islands,412,412,0,None,None,None,None,None
208,ip,Clipperton Island,6,6,0,None,None,None,None,None
210,fs,French Southern and Antarctic Lands,None,None,None,None,None,None,None,None


## Identifying non-Countries

In [9]:
%%sql
ALTER TABLE facts
        ADD is_a_country BOOL
    DEFAULT 1

 * sqlite:///factbook.db
Done.


[]

In [10]:
%%sql
SELECT *
  FROM facts
 WHERE name IN ('World', 'Holy See (Vatican City)','European Union')

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,is_a_country
190,vt,Holy See (Vatican City),0,0,0,842,0.0,None,None,None,1
197,ee,European Union,4324782,None,None,513949445,0.25,10.2,10.2,2.5,1
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None,1


In [11]:
%%sql
UPDATE facts
   SET is_a_country = 0
 WHERE population IS NULL
--     OR (population IS NOT NULL 
--         AND population_growth IS NULL 
--        AND birth_rate IS NULL 
--        AND death_rate IS NULL)
    OR name IN ('World', 'Holy See (Vatican City)', 'European Union')

 * sqlite:///factbook.db
22 rows affected.


[]

In [12]:
%%sql
UPDATE facts 
   SET area = 25, area_land = 25
 WHERE name = 'Saint Barthelemy'

 * sqlite:///factbook.db
1 rows affected.


[]

Saint Barthelemy has an area of 25 square km but is erroneously shown with a null value.

In [13]:
%%sql
SELECT * FROM facts
WHERE is_a_country = 1
  AND (area IS NULL OR area_land IS NULL)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,is_a_country


It doesn't make sense for a location to have land area but not area. We fill in the blanks by totaling land area and water area when appropriate or setting the land area equal to the area and vice versa if one is not provided.

In [14]:
%%sql
UPDATE facts 
   SET area = area_land + area_water WHERE name IN ('Chad', 'Niger')

 * sqlite:///factbook.db
2 rows affected.


[]

In [15]:
%%sql
UPDATE facts
   SET area_land = area - area_water WHERE name IN ('Ethiopia')

 * sqlite:///factbook.db
1 rows affected.


[]

In [16]:
%%sql
UPDATE facts
   SET area_land = area WHERE name IN ('South Sudan', 'Sudan', 'Akrotiri', 'Dhekelia')

 * sqlite:///factbook.db
4 rows affected.


[]

In [17]:
%%sql
UPDATE facts
  SET area = area_land WHERE name IN ('Antarctica')

 * sqlite:///factbook.db
1 rows affected.


[]

In [18]:
%%sql
SELECT COUNT(*) FROM facts
WHERE is_a_country = 1

 * sqlite:///factbook.db
Done.


COUNT(*)
239


We removed the most obvious locations like `World`, `Vatican City` and `European Union` as well as sparse islands and main oceans that are not countries. While there are far less countries than the 239 locations we narrowed the analysis down to, we now have a dataset with locations that are either countries or territories inhabited by people.

In [19]:
%%sql
SELECT * FROM facts
WHERE is_a_country = 1 AND area_water IS NULL

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,is_a_country
126,nz,New Zealand,267710,267710,None,4438393,0.82,13.33,7.36,2.21,1
162,od,South Sudan,644329,644329,None,12042910,4.02,36.91,8.18,11.47,1
165,su,Sudan,1861484,1861484,None,36108853,1.72,29.19,7.66,4.29,1
207,gl,Greenland,2166086,2166086,None,57733,0.0,14.48,8.49,5.98,1
212,tb,Saint Barthelemy,25,25,None,7237,None,None,None,None,1
213,rn,Saint Martin,54,54,None,31754,None,None,None,None,1
225,ax,Akrotiri,123,123,None,15700,None,None,None,None,1
231,dx,Dhekelia,130,130,None,15700,None,None,None,None,1
250,ay,Antarctica,280000,280000,None,0,None,None,None,None,1


## Exploring Outliers

In [20]:
%%sql
SELECT name
  FROM facts
 WHERE population = (SELECT min(population) 
                       FROM facts)
   AND is_a_country = 1

 * sqlite:///factbook.db
Done.


name
Antarctica


It makes sense that Antartica would show a zero population count.

In [21]:
%%sql
SELECT min(population) AS min_population,
       max(population) AS max_population,
       min(population_growth) AS min_population_growth,
       max(population_growth) AS max_population_growth
  FROM facts
 WHERE is_a_country = 1

 * sqlite:///factbook.db
Done.


min_population,max_population,min_population_growth,max_population_growth
0,1367485388,0.0,4.02


The new maximum population count is now 1,367,485,388 people.

In [22]:
%%sql
SELECT name
  FROM facts
 WHERE population = (SELECT max(population) 
                       FROM facts WHERE is_a_country = 1)

 * sqlite:///factbook.db
Done.


name
China


We see that the country with the highest population count is China.

## Finding Densely Populated Countries

We would like to know the most densely populated countries. In order to do that, we have to set a baseline comparison so we calculate what the average population and average area size are for all the countries.

In [23]:
%%sql
SELECT ROUND(AVG(population), 2) AS avg_population, 
       ROUND(AVG(area_land), 2) AS avg_area_land,
       ROUND(CAST(AVG(population) AS FLOAT) / AVG(area_land), 2) AS avg_population_density
  FROM facts
 WHERE is_a_country = 1

 * sqlite:///factbook.db
Done.


avg_population,avg_area_land,avg_population_density
30362060.07,552661.45,54.94


The average population count is 30,362,060 people and the average land area is 552,661 square kilometers.

We calculate the average population density to be 54.94 persons per square kilometers.

In [24]:
%%sql
SELECT name, 
       ROUND(CAST(population AS FLOAT)/area_land, 2) AS population_density,
       area_land
  FROM facts
 ORDER BY population_density DESC 
 LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population_density,area_land
Macau,21168.96,28
Monaco,15267.5,2
Singapore,8259.78,687
Hong Kong,6655.27,1073
Gaza Strip,5191.82,360


In [25]:
21168/54.94

385.29304696032034

In [26]:
15267/54.94

277.8849654168184

The most crowded country is Macau, with a population density 385 times the average. Monaco comes in second with 278 times the average.

In [27]:
%%sql
SELECT name,
       ROUND(CAST(population AS FLOAT)/area_land, 2) AS population_density,
       area_land
  FROM facts
 WHERE is_a_country = 1
 ORDER BY population_density  
 LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population_density,area_land
Antarctica,0.0,280000
Greenland,0.03,2166086
Svalbard,0.03,62045
Falkland Islands (Islas Malvinas),0.28,12173
Pitcairn Islands,1.02,47


The least densely populated countries are Antartica, Greenland and Svalbard. Antartica does not have a native population. 80% of Greenland is covered in inhabitable permanent ice sheets while Svalbard is made up of 400 partially inhabitable islands.

## Finding Countries with High Population-to-Area Ratio 

We want to see if countries are densely populated because of geographic reasons, whether the land area cannot meet the population demand.

In [28]:
%%sql
SELECT name,
       population/area as population_to_area,
       area,
       area_land
  FROM facts
 WHERE is_a_country = 1
 GROUP BY name
 ORDER BY population_to_area DESC
 LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population_to_area,area,area_land
Macau,21168,28,28
Monaco,15267,2,2
Singapore,8141,697,687
Hong Kong,6445,1108,1073
Gaza Strip,5191,360,360


The top five most densely populated (highest population-to-land-area) countries are also the countries with the highest population-to-area ratios. This is due to the land area for these countries being close to, if not the same, as the total area. We note that a few of these are not countries, so we refer to them as locations.

## Finding Countries with the Largest Population

In [29]:
%%sql
SELECT name, 
       population,
       population_growth
  FROM facts
 WHERE is_a_country = 1
 ORDER BY population DESC 
 LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population,population_growth
China,1367485388,0.45
India,1251695584,1.22
United States,321368864,0.78
Indonesia,255993674,0.92
Brazil,204259812,0.77


The five countries with the most people (in order) are China, India, the EU, United States, and Indonesia.

## Finding Countries with the Highest Growth Rate

In [30]:
%%sql
SELECT name, 
       population_growth,
       population
  FROM facts
 WHERE is_a_country = 1
 ORDER BY population_growth DESC 
 LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population_growth,population
South Sudan,4.02,12042910
Malawi,3.32,17964697
Burundi,3.28,10742276
Niger,3.25,18045729
Uganda,3.24,37101745


The five countries with the highest growth rate people (in order) are South Sudan, Malawi, Burundi, Niger, and Uganda.

## Finding Countries with Highest Ratios of Land to Water

In [31]:
%%sql
SELECT name, 
       area_land,
       area_water,
       CAST(area_land AS FLOAT)/area_water AS land_to_water_ratio
  FROM facts
 WHERE is_a_country = 1
 ORDER BY land_to_water_ratio DESC 
 LIMIT 5;

 * sqlite:///factbook.db
Done.


name,area_land,area_water,land_to_water_ratio
Bosnia and Herzegovina,51187,10,5118.7
Niger,1266700,300,4222.333333333333
Morocco,446300,250,1785.2
Guinea,245717,140,1755.1214285714286
Costa Rica,51060,40,1276.5


Note that while the above are the top five countries with the highest land-to-water ratio, it does not necessarily mean they are landlocked. Bosnia and Herzegovina has a unique geography in that most of its land closest to the shoreline is occupied by Croatia, making it almost landlocked. Niger, Morocco, and Guinea all have coastlines with a sizeable access to the Atlantic Ocean. Costa Rica is almost entirely surrounded by ocean.

## Finding Countries with Lowest Ratios of Land to Water

In [32]:
%%sql
SELECT name, 
       area_land,
       area_water,
       CAST(area_land AS FLOAT)/area_water AS land_to_water_ratio
  FROM facts
 WHERE is_a_country = 1
   AND area_water IS NOT NULL
 ORDER BY land_to_water_ratio, area_land DESC
 LIMIT 5;

 * sqlite:///factbook.db
Done.


name,area_land,area_water,land_to_water_ratio
Algeria,2381741,0,None
Saudi Arabia,2149690,0,None
Libya,1759540,0,None
Angola,1246700,0,None
Mauritania,1030700,0,None


The countries made up only of land (in descending order by land area) are Algeria, Saudi Arabia, Libya, Angola, and Mauritania. As expected, these are located in desert regions.

## What countries will add the most people to their populations next year?

In [33]:
%%sql
SELECT name, 
       population,
       population_growth,
       CAST(population*population_growth/100 AS INT) AS annual_growth
  FROM facts
 WHERE is_a_country = 1
 ORDER BY annual_growth DESC
 LIMIT 5;

 * sqlite:///factbook.db
Done.


name,population,population_growth,annual_growth
India,1251695584,1.22,15270686
China,1367485388,0.45,6153684
Nigeria,181562056,2.45,4448270
Pakistan,199085847,1.46,2906653
Ethiopia,99465819,2.89,2874562


While the countries listed above do not have the highest population growth rates, both China and India are ranked one and two out of the countries with the highest population with over 1 billion people each. We can expect to see India add 15 million people next year, as many as the total expected population growth of China, Nigeria, Pakistan, and Ethiopia combined.

## What countries have a natural decrease?

In [34]:
%%sql
SELECT name, 
       birth_rate,
       death_rate
  FROM facts
 WHERE is_a_country = 1
   AND death_rate > birth_rate
 ORDER BY death_rate DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate
Ukraine,10.72,14.46
Bulgaria,8.92,14.44
Latvia,10.0,14.31
Lithuania,10.1,14.27
Russia,11.6,13.69
Serbia,9.08,13.66
Belarus,10.7,13.36
Hungary,9.16,12.73
Moldova,12.0,12.59
Estonia,10.51,12.4


## What countries have a natural increase?

It appears that many European countries are afflicted with natural decrease, the highest being Ukraine, Bulgaria, Latvia, Lithuania, and Russia.

In [35]:
%%sql
SELECT name, 
       birth_rate,
       death_rate
  FROM facts
 WHERE is_a_country = 1
   AND death_rate < birth_rate
 ORDER BY birth_rate DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate
Niger,45.45,12.42
Mali,44.99,12.89
Uganda,43.79,10.69
Zambia,42.13,12.67
Burkina Faso,42.03,11.72
Burundi,42.01,9.27
Malawi,41.56,8.41
Somalia,40.45,13.62
Angola,38.78,11.49
Mozambique,38.58,12.1


It appears that many African countries are afflicted with natural increase, the highest being Niger, Mali, Uganda, Zambia, and Burkina Faso.